In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import qiskit as qk
import qiskit.visualization

from qiskit import Aer
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit import transpile
from qiskit.quantum_info.operators import Operator

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.python.framework.ops import convert_to_tensor

TensorFlow version: 2.11.0


In [2]:
#Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

## Data Generation + (In)Fidelity

In [3]:
# Generation of a random unitary transformation
def random_unitary(N):
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [4]:
#Function which generates the density matrix given the state vector
def get_density_matrix(state_vector):
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [5]:
#Function which computes the components of the Bloch vector, given the density matrix 

#Here we define the identity matrix and the Pauli matrices for dimension 2 (one qubit)
I = np.array([[1, 0],[0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

def Bloch_vector(rho):
    ax = np.trace(np.dot(rho, X)).real
    ay = np.trace(np.dot(rho, Y)).real
    az = np.trace(np.dot(rho, Z)).real
    pnt = [ax, ay, az]
    return np.array(pnt)

In [22]:
def density_matrix_from_bloch_vector(bloch_vector):
   rho = 0.5 * (I+ bloch_vector[0]*X + bloch_vector[1]*Y + bloch_vector[2]*Z)
   return rho

In [20]:
def fidelity_function(a,b):
    fid = tf.linalg.trace(a @ b)
    return fid

In [27]:
I_tf = tf.constant([1.0, 0.0,0.0,1.0],shape=(2,2), dtype = tf.complex64)
X_tf = tf.constant([0.0, 1.0, 1.0, 0.0],shape=(2,2), dtype = tf.complex64)
Y_tf = tf.constant([0.0+0j, 0.0-1j ,0.0+1j,0.0+0j],shape=(2,2), dtype = tf.complex64)
Z_tf = tf.constant([1.0, 0.0,0.0,-1.0],shape=(2,2), dtype = tf.complex64)
A = tf.stack([X_tf,Y_tf,Z_tf]) 

In [28]:
@tf.function
def infidelity(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk',A,a) 
   el_b = tf.einsum('ijk,mi->mjk',A,b) 
   rho_a = 0.5 *(el_a + I_tf)
   rho_b = 0.5 * (el_b +I_tf)
   fidelity = tf.linalg.trace(rho_a @ rho_b) 
   infidelity = 1 - fidelity
   return infidelity 

In [6]:
#Function to represent the data on the Bloch sphere

def Bloch_sphere(data):
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(projection='3d')
    qk.visualization.plot_bloch_vector([0, 0, 0], title='Bloch Sphere',ax=ax)
    for (x, y, z) in data:
        ax.scatter3D(y, -x, z, c='b') #here x and y axis are inverted in order to match qiskit and matplotlib axis

In [7]:
sim_bknd=Aer.get_backend('aer_simulator')

In [8]:
def generate_Haar_data(num_qubits, samples=1000):
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) #creates a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) #applies the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) #it's used to optimize the circuit
        qc.save_statevector() #it's the instruction to save the state vector obtained by the simulation

        state = sim_bknd.run(qc).result().get_statevector(qc) #does the simulation and gets the state vector
        state = np.asarray(state)
        data.append(state) 
    return data

## Noise Model

In [9]:
#SINGLE ELEMENT OF THE OPERATOR-SUM REPRESENTATION

def sum_element(rho,operator):
    element = np.dot(np.dot(operator,rho),operator.conj().T)
    return element

In [10]:
#BIT FLIP, PHASE FLIP AND BIT-PHASE FLIP ERROR

def flip_error(num_qubits, rho, error_type, p):

    E_0 = np.sqrt(1-p)*I
    if error_type == 'bit':
        E_1 = np.sqrt(p)*X
    if error_type == 'phase':
        E_1 = np.sqrt(p)*Z
    if error_type == 'bp':
        E_1 = np.sqrt(p)*Y
        
    rho_with_flip_error = sum_element(rho, E_0)+sum_element(rho, E_1)
    
    return rho_with_flip_error

In [11]:
#GENERAL PAULI ERROR (generalizes and allows to combine the flip errors)

def g_pauli_error(num_qubits, rho, p_0, p_1, p_2, p_3):
    
    E_0 = np.sqrt(p_0)*I
    E_1 = np.sqrt(p_1)*X
    E_2 = np.sqrt(p_2)*Y
    E_3 = np.sqrt(p_3)*Z
    
    rho_with_gpauli_error = sum_element(rho, E_0)+sum_element(rho, E_1)+sum_element(rho, E_2)+sum_element(rho, E_3)
    
    return rho_with_gpauli_error

In [12]:
# DEPOLARIZING ERROR

def depolar_error(num_qubits, rho, p):
    
    rho_with_dep_error = p*I/2 + (1-p)*rho

    return rho_with_dep_error

In [13]:
#GENERAL AMPLITUDE DAMPING ERROR

def ampl_damp_error(num_qubits, rho, p, a):
  
    E_0 = np.sqrt(p)*np.array([[1, 0], [0, np.sqrt(1-a)]])
    E_1 = np.sqrt(p)*np.array([[0, np.sqrt(a)], [0, 0]])
    E_2 = np.sqrt(1-p)*np.array([[np.sqrt(1-a), 0], [0, 1]])
    E_3 = np.sqrt(1-p)*np.array([[0, 0], [np.sqrt(a), 0]])
    
    rho_with_ad_error = sum_element(rho, E_0)+sum_element(rho, E_1)+sum_element(rho, E_2)+sum_element(rho, E_3)

    return rho_with_ad_error

## Regression Problem

In [15]:
# GENERATE DATA
data = generate_Haar_data(1, 100)
#COMPUTE NOISE FREE DENSITY MATRIX
density_matrix_noise_free = [*map(get_density_matrix, data)]
#COMPUTE BLOCH VECTOR NOISE FREE
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

#PRINT THE TYPE OF THE SINGLE DENSITY MATRIX IN THE LISTS OF THE DENSITY MATRICES NOISE FREE
print("Density matrix noise free type:", density_matrix_noise_free[0].dtype)
#PRINT THE TYPE OF THE BLOCH VECTOR IN THE LISTS OF THE BLOCH VECTORS NOISE FREE
print("Bloch vector noise free type:", bloch_vectors_noise_free[0].dtype)

Density matrix noise free type: complex128
Bloch vector noise free type: float64


## Phase Flip Channel

In [16]:
#APPLY ERROR (in this case phase error with p=0.2)
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)

#PRINT THE TYPE OF THE SINGLE DENSITY MATRIX IN THE LISTS OF THE DENSITY MATRICES WITH NOISE
print("Density matrix with noise type:", density_matrix_with_noise[0].dtype)

#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
print("Bloch vector with noise type:", bloch_vectors_with_noise[0].dtype)

Density matrix with noise type: complex128
Bloch vector with noise type: float64


In [17]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)
print("Input data type:", x_train.dtype)
print("Output data type:", y_train.dtype)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

Input data type: <dtype: 'float64'>
Output data type: <dtype: 'float64'>


### MSE

In [18]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
  ])

In [19]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=loss_fn)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 94ms/step - loss: 0.3024 - val_loss: 0.2616
Epoch 2/200
5/5 [==============================] - 0s 22ms/step - loss: 0.2572 - val_loss: 0.2206
Epoch 3/200
5/5 [==============================] - 0s 15ms/step - loss: 0.2155 - val_loss: 0.1825
Epoch 4/200
5/5 [==============================] - 0s 15ms/step - loss: 0.1782 - val_loss: 0.1483
Epoch 5/200
5/5 [==============================] - 0s 24ms/step - loss: 0.1436 - val_loss: 0.1173
Epoch 6/200
5/5 [==============================] - 0s 27ms/step - loss: 0.1122 - val_loss: 0.0897
Epoch 7/200
5/5 [==============================] - 0s 18ms/step - loss: 0.0850 - val_loss: 0.0664
Epoch 8/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0604 - val_loss: 0.0469
Epoch 9/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0410 - val_loss: 0.0313
Epoch 10/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0253 - val_loss: 0.0192
Epoch 11/200
5/5 [=

Epoch 79/200
5/5 [==============================] - 0s 14ms/step - loss: 5.1328e-05 - val_loss: 3.6075e-04
Epoch 80/200
5/5 [==============================] - 0s 14ms/step - loss: 5.0489e-05 - val_loss: 3.5868e-04
Epoch 81/200
5/5 [==============================] - 0s 18ms/step - loss: 4.8426e-05 - val_loss: 3.5407e-04
Epoch 82/200
5/5 [==============================] - 0s 19ms/step - loss: 4.9086e-05 - val_loss: 3.5923e-04
Epoch 83/200
5/5 [==============================] - 0s 26ms/step - loss: 4.6254e-05 - val_loss: 3.5754e-04
Epoch 84/200
5/5 [==============================] - 0s 20ms/step - loss: 4.5949e-05 - val_loss: 3.4923e-04
Epoch 85/200
5/5 [==============================] - 0s 16ms/step - loss: 4.4547e-05 - val_loss: 3.5257e-04
Epoch 86/200
5/5 [==============================] - 0s 15ms/step - loss: 4.4451e-05 - val_loss: 3.5292e-04
Epoch 87/200
5/5 [==============================] - 0s 16ms/step - loss: 4.2161e-05 - val_loss: 3.4672e-04
Epoch 88/200
5/5 [===================

5/5 [==============================] - 0s 17ms/step - loss: 1.2497e-05 - val_loss: 3.0476e-04
Epoch 156/200
5/5 [==============================] - 0s 16ms/step - loss: 1.2132e-05 - val_loss: 3.0771e-04
Epoch 157/200
5/5 [==============================] - 0s 16ms/step - loss: 1.2069e-05 - val_loss: 2.9673e-04
Epoch 158/200
5/5 [==============================] - 0s 16ms/step - loss: 1.1574e-05 - val_loss: 3.0372e-04
Epoch 159/200
5/5 [==============================] - 0s 20ms/step - loss: 1.1574e-05 - val_loss: 3.0327e-04
Epoch 160/200
5/5 [==============================] - 0s 15ms/step - loss: 1.1545e-05 - val_loss: 3.0014e-04
Epoch 161/200
5/5 [==============================] - 0s 17ms/step - loss: 1.0540e-05 - val_loss: 3.0418e-04
Epoch 162/200
5/5 [==============================] - 0s 19ms/step - loss: 1.1249e-05 - val_loss: 2.9981e-04
Epoch 163/200
5/5 [==============================] - 0s 18ms/step - loss: 1.0853e-05 - val_loss: 3.0023e-04
Epoch 164/200
5/5 [=======================

In [25]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 1.3648e-04 - 40ms/epoch - 40ms/step
tf.Tensor((0.997434203101137+0j), shape=(), dtype=complex128)


### INF

In [26]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
  ])

In [29]:
# Define Loss
adam_opt = tf.optimizers.Adam(0.0001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=300)

Epoch 1/300
5/5 [==============================] - 2s 82ms/step - loss: 0.4958 - val_loss: 0.5261
Epoch 2/300
5/5 [==============================] - 0s 14ms/step - loss: 0.4729 - val_loss: 0.5108
Epoch 3/300
5/5 [==============================] - 0s 14ms/step - loss: 0.4543 - val_loss: 0.4955
Epoch 4/300
5/5 [==============================] - 0s 13ms/step - loss: 0.4376 - val_loss: 0.4810
Epoch 5/300
5/5 [==============================] - 0s 24ms/step - loss: 0.4220 - val_loss: 0.4672
Epoch 6/300
5/5 [==============================] - 0s 20ms/step - loss: 0.4084 - val_loss: 0.4525
Epoch 7/300
5/5 [==============================] - 0s 16ms/step - loss: 0.3941 - val_loss: 0.4374
Epoch 8/300
5/5 [==============================] - 0s 14ms/step - loss: 0.3785 - val_loss: 0.4211
Epoch 9/300
5/5 [==============================] - 0s 17ms/step - loss: 0.3630 - val_loss: 0.4028
Epoch 10/300
5/5 [==============================] - 0s 20ms/step - loss: 0.3449 - val_loss: 0.3824
Epoch 11/300
5/5 [=

Epoch 84/300
5/5 [==============================] - 0s 22ms/step - loss: 0.0027 - val_loss: 0.0083
Epoch 85/300
5/5 [==============================] - 0s 20ms/step - loss: 0.0026 - val_loss: 0.0082
Epoch 86/300
5/5 [==============================] - 0s 16ms/step - loss: 0.0025 - val_loss: 0.0080
Epoch 87/300
5/5 [==============================] - 0s 16ms/step - loss: 0.0025 - val_loss: 0.0079
Epoch 88/300
5/5 [==============================] - 0s 19ms/step - loss: 0.0024 - val_loss: 0.0077
Epoch 89/300
5/5 [==============================] - 0s 16ms/step - loss: 0.0023 - val_loss: 0.0076
Epoch 90/300
5/5 [==============================] - 0s 14ms/step - loss: 0.0023 - val_loss: 0.0075
Epoch 91/300
5/5 [==============================] - 0s 16ms/step - loss: 0.0022 - val_loss: 0.0073
Epoch 92/300
5/5 [==============================] - 0s 21ms/step - loss: 0.0022 - val_loss: 0.0072
Epoch 93/300
5/5 [==============================] - 0s 20ms/step - loss: 0.0021 - val_loss: 0.0071
Epoch 94/3

5/5 [==============================] - 0s 14ms/step - loss: 5.6502e-04 - val_loss: 0.0036
Epoch 165/300
5/5 [==============================] - 0s 14ms/step - loss: 5.5381e-04 - val_loss: 0.0035
Epoch 166/300
5/5 [==============================] - 0s 14ms/step - loss: 5.4603e-04 - val_loss: 0.0035
Epoch 167/300
5/5 [==============================] - 0s 15ms/step - loss: 5.4022e-04 - val_loss: 0.0035
Epoch 168/300
5/5 [==============================] - 0s 16ms/step - loss: 5.3346e-04 - val_loss: 0.0035
Epoch 169/300
5/5 [==============================] - 0s 14ms/step - loss: 5.2555e-04 - val_loss: 0.0034
Epoch 170/300
5/5 [==============================] - 0s 14ms/step - loss: 5.1841e-04 - val_loss: 0.0034
Epoch 171/300
5/5 [==============================] - 0s 16ms/step - loss: 5.1237e-04 - val_loss: 0.0034
Epoch 172/300
5/5 [==============================] - 0s 16ms/step - loss: 5.0580e-04 - val_loss: 0.0034
Epoch 173/300
5/5 [==============================] - 0s 16ms/step - loss: 4.98

Epoch 243/300
5/5 [==============================] - 0s 17ms/step - loss: 2.3648e-04 - val_loss: 0.0026
Epoch 244/300
5/5 [==============================] - 0s 14ms/step - loss: 2.3299e-04 - val_loss: 0.0026
Epoch 245/300
5/5 [==============================] - 0s 16ms/step - loss: 2.3172e-04 - val_loss: 0.0026
Epoch 246/300
5/5 [==============================] - 0s 16ms/step - loss: 2.2963e-04 - val_loss: 0.0026
Epoch 247/300
5/5 [==============================] - 0s 14ms/step - loss: 2.2595e-04 - val_loss: 0.0026
Epoch 248/300
5/5 [==============================] - 0s 14ms/step - loss: 2.2759e-04 - val_loss: 0.0026
Epoch 249/300
5/5 [==============================] - 0s 14ms/step - loss: 2.2378e-04 - val_loss: 0.0026
Epoch 250/300
5/5 [==============================] - 0s 14ms/step - loss: 2.2026e-04 - val_loss: 0.0026
Epoch 251/300
5/5 [==============================] - 0s 16ms/step - loss: 2.1962e-04 - val_loss: 0.0026
Epoch 252/300
5/5 [==============================] - 0s 14ms/ste

In [30]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 0.0022 - 202ms/epoch - 202ms/step
tf.Tensor((0.9977837759336639+0j), shape=(), dtype=complex128)


## Bit Flip Channel

In [31]:
#APPLY ERROR (in this case phase error with p=0.2)
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'bit', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)

#PRINT THE TYPE OF THE SINGLE DENSITY MATRIX IN THE LISTS OF THE DENSITY MATRICES WITH NOISE
print("Density matrix with noise type:", density_matrix_with_noise[0].dtype)

#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
print("Bloch vector with noise type:", bloch_vectors_with_noise[0].dtype)

Density matrix with noise type: complex128
Bloch vector with noise type: float64


In [32]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)
print("Input data type:", x_train.dtype)
print("Output data type:", y_train.dtype)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

Input data type: <dtype: 'float64'>
Output data type: <dtype: 'float64'>


### MSE

In [33]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
  ])

In [34]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=loss_fn)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 69ms/step - loss: 0.3429 - val_loss: 0.3086
Epoch 2/200
5/5 [==============================] - 0s 28ms/step - loss: 0.2874 - val_loss: 0.2668
Epoch 3/200
5/5 [==============================] - 0s 22ms/step - loss: 0.2430 - val_loss: 0.2285
Epoch 4/200
5/5 [==============================] - 0s 19ms/step - loss: 0.2053 - val_loss: 0.1948
Epoch 5/200
5/5 [==============================] - 0s 14ms/step - loss: 0.1718 - val_loss: 0.1638
Epoch 6/200
5/5 [==============================] - 0s 16ms/step - loss: 0.1409 - val_loss: 0.1345
Epoch 7/200
5/5 [==============================] - 0s 16ms/step - loss: 0.1134 - val_loss: 0.1085
Epoch 8/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0882 - val_loss: 0.0847
Epoch 9/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0662 - val_loss: 0.0646
Epoch 10/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0493 - val_loss: 0.0469
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 14ms/step - loss: 4.5879e-05 - val_loss: 2.5397e-04
Epoch 80/200
5/5 [==============================] - 0s 15ms/step - loss: 4.4824e-05 - val_loss: 2.5865e-04
Epoch 81/200
5/5 [==============================] - 0s 14ms/step - loss: 4.3545e-05 - val_loss: 2.5134e-04
Epoch 82/200
5/5 [==============================] - 0s 14ms/step - loss: 4.2035e-05 - val_loss: 2.4456e-04
Epoch 83/200
5/5 [==============================] - 0s 15ms/step - loss: 4.1042e-05 - val_loss: 2.4881e-04
Epoch 84/200
5/5 [==============================] - 0s 14ms/step - loss: 3.9897e-05 - val_loss: 2.4450e-04
Epoch 85/200
5/5 [==============================] - 0s 14ms/step - loss: 3.9623e-05 - val_loss: 2.4317e-04
Epoch 86/200
5/5 [==============================] - 0s 14ms/step - loss: 3.8890e-05 - val_loss: 2.4626e-04
Epoch 87/200
5/5 [==============================] - 0s 14ms/step - loss: 3.7944e-05 - val_loss: 2.4501e-04
Epoch 88/200
5/5 [==============================] 

5/5 [==============================] - 0s 14ms/step - loss: 1.1087e-05 - val_loss: 2.2054e-04
Epoch 156/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0583e-05 - val_loss: 2.2267e-04
Epoch 157/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0647e-05 - val_loss: 2.2062e-04
Epoch 158/200
5/5 [==============================] - 0s 22ms/step - loss: 1.0645e-05 - val_loss: 2.1751e-04
Epoch 159/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0363e-05 - val_loss: 2.2352e-04
Epoch 160/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0463e-05 - val_loss: 2.2234e-04
Epoch 161/200
5/5 [==============================] - 0s 16ms/step - loss: 1.0147e-05 - val_loss: 2.2303e-04
Epoch 162/200
5/5 [==============================] - 0s 18ms/step - loss: 9.7906e-06 - val_loss: 2.2045e-04
Epoch 163/200
5/5 [==============================] - 0s 16ms/step - loss: 1.0918e-05 - val_loss: 2.1741e-04
Epoch 164/200
5/5 [=======================

In [35]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 3.8901e-05 - 153ms/epoch - 153ms/step
tf.Tensor((1.0004115316502333+0j), shape=(), dtype=complex128)


### INF

In [39]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
  ])

In [40]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 66ms/step - loss: 0.5121 - val_loss: 0.4123
Epoch 2/200
5/5 [==============================] - 0s 18ms/step - loss: 0.3544 - val_loss: 0.3150
Epoch 3/200
5/5 [==============================] - 0s 16ms/step - loss: 0.2476 - val_loss: 0.2123
Epoch 4/200
5/5 [==============================] - 0s 16ms/step - loss: 0.1494 - val_loss: 0.1351
Epoch 5/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0905 - val_loss: 0.0915
Epoch 6/200
5/5 [==============================] - 0s 18ms/step - loss: 0.0599 - val_loss: 0.0604
Epoch 7/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0377 - val_loss: 0.0392
Epoch 8/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0237 - val_loss: 0.0255
Epoch 9/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0147 - val_loss: 0.0173
Epoch 10/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0099 - val_loss: 0.0125
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 14ms/step - loss: 4.0755e-05 - val_loss: 5.8287e-04
Epoch 81/200
5/5 [==============================] - 0s 16ms/step - loss: 3.9818e-05 - val_loss: 5.7980e-04
Epoch 82/200
5/5 [==============================] - 0s 16ms/step - loss: 3.8283e-05 - val_loss: 5.7962e-04
Epoch 83/200
5/5 [==============================] - 0s 14ms/step - loss: 3.7174e-05 - val_loss: 5.7567e-04
Epoch 84/200
5/5 [==============================] - 0s 13ms/step - loss: 3.6293e-05 - val_loss: 5.7654e-04
Epoch 85/200
5/5 [==============================] - 0s 14ms/step - loss: 3.6297e-05 - val_loss: 5.7184e-04
Epoch 86/200
5/5 [==============================] - 0s 14ms/step - loss: 3.3991e-05 - val_loss: 5.7062e-04
Epoch 87/200
5/5 [==============================] - 0s 15ms/step - loss: 3.4351e-05 - val_loss: 5.6340e-04
Epoch 88/200
5/5 [==============================] - 0s 22ms/step - loss: 3.2569e-05 - val_loss: 5.5516e-04
Epoch 89/200
5/5 [==============================] 

5/5 [==============================] - 0s 23ms/step - loss: 8.7953e-06 - val_loss: 4.7150e-04
Epoch 157/200
5/5 [==============================] - 0s 26ms/step - loss: 8.7190e-06 - val_loss: 4.7143e-04
Epoch 158/200
5/5 [==============================] - 0s 24ms/step - loss: 9.1040e-06 - val_loss: 4.6678e-04
Epoch 159/200
5/5 [==============================] - 0s 16ms/step - loss: 9.1267e-06 - val_loss: 4.7064e-04
Epoch 160/200
5/5 [==============================] - 0s 25ms/step - loss: 8.2123e-06 - val_loss: 4.7285e-04
Epoch 161/200
5/5 [==============================] - 0s 25ms/step - loss: 8.1086e-06 - val_loss: 4.6868e-04
Epoch 162/200
5/5 [==============================] - 0s 24ms/step - loss: 8.4603e-06 - val_loss: 4.6540e-04
Epoch 163/200
5/5 [==============================] - 0s 16ms/step - loss: 8.3995e-06 - val_loss: 4.6745e-04
Epoch 164/200
5/5 [==============================] - 0s 16ms/step - loss: 8.3232e-06 - val_loss: 4.6865e-04
Epoch 165/200
5/5 [=======================

In [41]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 5.2003e-04 - 176ms/epoch - 176ms/step
tf.Tensor((0.9994799637530694+0j), shape=(), dtype=complex128)


## Bit-Phase Flip Channel

In [42]:
#APPLY ERROR (in this case phase error with p=0.2)
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'bp', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)

#PRINT THE TYPE OF THE SINGLE DENSITY MATRIX IN THE LISTS OF THE DENSITY MATRICES WITH NOISE
print("Density matrix with noise type:", density_matrix_with_noise[0].dtype)

#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
print("Bloch vector with noise type:", bloch_vectors_with_noise[0].dtype)

Density matrix with noise type: complex128
Bloch vector with noise type: float64


In [43]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)
print("Input data type:", x_train.dtype)
print("Output data type:", y_train.dtype)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

Input data type: <dtype: 'float64'>
Output data type: <dtype: 'float64'>


### MSE

In [44]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
  ])

In [45]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=loss_fn)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 71ms/step - loss: 0.3324 - val_loss: 0.3043
Epoch 2/200
5/5 [==============================] - 0s 20ms/step - loss: 0.2832 - val_loss: 0.2675
Epoch 3/200
5/5 [==============================] - 0s 21ms/step - loss: 0.2407 - val_loss: 0.2322
Epoch 4/200
5/5 [==============================] - 0s 20ms/step - loss: 0.2026 - val_loss: 0.2000
Epoch 5/200
5/5 [==============================] - 0s 25ms/step - loss: 0.1681 - val_loss: 0.1689
Epoch 6/200
5/5 [==============================] - 0s 25ms/step - loss: 0.1348 - val_loss: 0.1394
Epoch 7/200
5/5 [==============================] - 0s 23ms/step - loss: 0.1054 - val_loss: 0.1126
Epoch 8/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0799 - val_loss: 0.0886
Epoch 9/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0589 - val_loss: 0.0687
Epoch 10/200
5/5 [==============================] - 0s 23ms/step - loss: 0.0436 - val_loss: 0.0524
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 20ms/step - loss: 7.3626e-05 - val_loss: 5.8636e-04
Epoch 81/200
5/5 [==============================] - 0s 18ms/step - loss: 7.1559e-05 - val_loss: 5.8307e-04
Epoch 82/200
5/5 [==============================] - 0s 18ms/step - loss: 7.0361e-05 - val_loss: 5.6688e-04
Epoch 83/200
5/5 [==============================] - 0s 20ms/step - loss: 7.0003e-05 - val_loss: 5.8226e-04
Epoch 84/200
5/5 [==============================] - 0s 19ms/step - loss: 6.6749e-05 - val_loss: 5.6230e-04
Epoch 85/200
5/5 [==============================] - 0s 21ms/step - loss: 6.4787e-05 - val_loss: 5.7038e-04
Epoch 86/200
5/5 [==============================] - 0s 20ms/step - loss: 6.3812e-05 - val_loss: 5.5774e-04
Epoch 87/200
5/5 [==============================] - 0s 18ms/step - loss: 6.1495e-05 - val_loss: 5.5682e-04
Epoch 88/200
5/5 [==============================] - 0s 19ms/step - loss: 6.2282e-05 - val_loss: 5.5375e-04
Epoch 89/200
5/5 [==============================] 

5/5 [==============================] - 0s 18ms/step - loss: 1.4238e-05 - val_loss: 4.2389e-04
Epoch 157/200
5/5 [==============================] - 0s 20ms/step - loss: 1.3733e-05 - val_loss: 4.3267e-04
Epoch 158/200
5/5 [==============================] - 0s 19ms/step - loss: 1.3725e-05 - val_loss: 4.2517e-04
Epoch 159/200
5/5 [==============================] - 0s 22ms/step - loss: 1.3328e-05 - val_loss: 4.2492e-04
Epoch 160/200
5/5 [==============================] - 0s 18ms/step - loss: 1.2849e-05 - val_loss: 4.2711e-04
Epoch 161/200
5/5 [==============================] - 0s 20ms/step - loss: 1.2883e-05 - val_loss: 4.1447e-04
Epoch 162/200
5/5 [==============================] - 0s 20ms/step - loss: 1.2700e-05 - val_loss: 4.1938e-04
Epoch 163/200
5/5 [==============================] - 0s 31ms/step - loss: 1.2690e-05 - val_loss: 4.2068e-04
Epoch 164/200
5/5 [==============================] - 0s 22ms/step - loss: 1.2360e-05 - val_loss: 4.1997e-04
Epoch 165/200
5/5 [=======================

In [46]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 6.8123e-05 - 167ms/epoch - 167ms/step
tf.Tensor((1.0020681254800978+0j), shape=(), dtype=complex128)


### INF

In [47]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
  ])

In [48]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 91ms/step - loss: 0.4027 - val_loss: 0.2550
Epoch 2/200
5/5 [==============================] - 0s 22ms/step - loss: 0.1404 - val_loss: 0.0674
Epoch 3/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0442 - val_loss: 0.0509
Epoch 4/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0286 - val_loss: 0.0317
Epoch 5/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0141 - val_loss: 0.0159
Epoch 6/200
5/5 [==============================] - 0s 23ms/step - loss: 0.0084 - val_loss: 0.0086
Epoch 7/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0060 - val_loss: 0.0059
Epoch 8/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0040 - val_loss: 0.0048
Epoch 9/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0027 - val_loss: 0.0047
Epoch 10/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0025 - val_loss: 0.0049
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 24ms/step - loss: 6.2581e-05 - val_loss: 0.0017
Epoch 82/200
5/5 [==============================] - 0s 26ms/step - loss: 5.5714e-05 - val_loss: 0.0017
Epoch 83/200
5/5 [==============================] - 0s 24ms/step - loss: 5.8045e-05 - val_loss: 0.0016
Epoch 84/200
5/5 [==============================] - 0s 24ms/step - loss: 5.4549e-05 - val_loss: 0.0017
Epoch 85/200
5/5 [==============================] - 0s 22ms/step - loss: 5.3746e-05 - val_loss: 0.0016
Epoch 86/200
5/5 [==============================] - 0s 20ms/step - loss: 5.0371e-05 - val_loss: 0.0016
Epoch 87/200
5/5 [==============================] - 0s 18ms/step - loss: 4.9696e-05 - val_loss: 0.0016
Epoch 88/200
5/5 [==============================] - 0s 24ms/step - loss: 4.7004e-05 - val_loss: 0.0017
Epoch 89/200
5/5 [==============================] - 0s 24ms/step - loss: 4.6179e-05 - val_loss: 0.0016
Epoch 90/200
5/5 [==============================] - 0s 24ms/step - loss: 4.3951e-05 - 

5/5 [==============================] - 0s 24ms/step - loss: 6.6495e-06 - val_loss: 0.0015
Epoch 161/200
5/5 [==============================] - 0s 24ms/step - loss: 6.2048e-06 - val_loss: 0.0015
Epoch 162/200
5/5 [==============================] - 0s 20ms/step - loss: 5.8949e-06 - val_loss: 0.0015
Epoch 163/200
5/5 [==============================] - 0s 20ms/step - loss: 6.2716e-06 - val_loss: 0.0015
Epoch 164/200
5/5 [==============================] - 0s 20ms/step - loss: 6.0892e-06 - val_loss: 0.0015
Epoch 165/200
5/5 [==============================] - 0s 18ms/step - loss: 5.4884e-06 - val_loss: 0.0015
Epoch 166/200
5/5 [==============================] - 0s 22ms/step - loss: 5.7578e-06 - val_loss: 0.0015
Epoch 167/200
5/5 [==============================] - 0s 26ms/step - loss: 5.5015e-06 - val_loss: 0.0014
Epoch 168/200
5/5 [==============================] - 0s 20ms/step - loss: 5.0211e-06 - val_loss: 0.0014
Epoch 169/200
5/5 [==============================] - 0s 21ms/step - loss: 5.22

In [49]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 3.4182e-04 - 234ms/epoch - 234ms/step
tf.Tensor((0.9996581697266729+0j), shape=(), dtype=complex128)


## General Pauli Channel

In [50]:
#APPLY ERROR 
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = g_pauli_error(1, density_matrix_noise_free[i], 0.7, 0.1, 0.1, 0.1)
    density_matrix_with_noise.append(single_data_with_noise)

#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

In [51]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

### MSE

In [52]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
  ])

In [53]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=loss_fn)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 65ms/step - loss: 0.2920 - val_loss: 0.2692
Epoch 2/200
5/5 [==============================] - 0s 23ms/step - loss: 0.2480 - val_loss: 0.2328
Epoch 3/200
5/5 [==============================] - 0s 30ms/step - loss: 0.2087 - val_loss: 0.1983
Epoch 4/200
5/5 [==============================] - 0s 22ms/step - loss: 0.1736 - val_loss: 0.1672
Epoch 5/200
5/5 [==============================] - 0s 20ms/step - loss: 0.1420 - val_loss: 0.1387
Epoch 6/200
5/5 [==============================] - 0s 18ms/step - loss: 0.1132 - val_loss: 0.1117
Epoch 7/200
5/5 [==============================] - 0s 21ms/step - loss: 0.0871 - val_loss: 0.0876
Epoch 8/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0641 - val_loss: 0.0656
Epoch 9/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0450 - val_loss: 0.0472
Epoch 10/200
5/5 [==============================] - 0s 28ms/step - loss: 0.0294 - val_loss: 0.0323
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 20ms/step - loss: 5.5657e-05 - val_loss: 4.2755e-04
Epoch 80/200
5/5 [==============================] - 0s 20ms/step - loss: 5.2683e-05 - val_loss: 4.3725e-04
Epoch 81/200
5/5 [==============================] - 0s 21ms/step - loss: 5.2619e-05 - val_loss: 4.3304e-04
Epoch 82/200
5/5 [==============================] - 0s 18ms/step - loss: 4.9871e-05 - val_loss: 4.2194e-04
Epoch 83/200
5/5 [==============================] - 0s 18ms/step - loss: 4.9994e-05 - val_loss: 4.2576e-04
Epoch 84/200
5/5 [==============================] - 0s 18ms/step - loss: 4.9861e-05 - val_loss: 4.2525e-04
Epoch 85/200
5/5 [==============================] - 0s 18ms/step - loss: 4.7065e-05 - val_loss: 4.2031e-04
Epoch 86/200
5/5 [==============================] - 0s 22ms/step - loss: 4.5870e-05 - val_loss: 4.1836e-04
Epoch 87/200
5/5 [==============================] - 0s 26ms/step - loss: 4.4520e-05 - val_loss: 4.1260e-04
Epoch 88/200
5/5 [==============================] 

5/5 [==============================] - 0s 21ms/step - loss: 1.0814e-05 - val_loss: 3.3775e-04
Epoch 156/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0196e-05 - val_loss: 3.3704e-04
Epoch 157/200
5/5 [==============================] - 0s 18ms/step - loss: 9.8121e-06 - val_loss: 3.4127e-04
Epoch 158/200
5/5 [==============================] - 0s 23ms/step - loss: 1.0068e-05 - val_loss: 3.3964e-04
Epoch 159/200
5/5 [==============================] - 0s 20ms/step - loss: 9.4875e-06 - val_loss: 3.3762e-04
Epoch 160/200
5/5 [==============================] - 0s 26ms/step - loss: 9.5047e-06 - val_loss: 3.3687e-04
Epoch 161/200
5/5 [==============================] - 0s 24ms/step - loss: 9.5284e-06 - val_loss: 3.4051e-04
Epoch 162/200
5/5 [==============================] - 0s 22ms/step - loss: 9.5228e-06 - val_loss: 3.3764e-04
Epoch 163/200
5/5 [==============================] - 0s 24ms/step - loss: 8.6555e-06 - val_loss: 3.3281e-04
Epoch 164/200
5/5 [=======================

In [54]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 8.2074e-05 - 217ms/epoch - 217ms/step
tf.Tensor((1.000344825592792+0j), shape=(), dtype=complex128)


### INF

In [55]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
  ])

In [56]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 90ms/step - loss: 0.4356 - val_loss: 0.3076
Epoch 2/200
5/5 [==============================] - 0s 20ms/step - loss: 0.2349 - val_loss: 0.1562
Epoch 3/200
5/5 [==============================] - 0s 18ms/step - loss: 0.1357 - val_loss: 0.0740
Epoch 4/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0605 - val_loss: 0.0442
Epoch 5/200
5/5 [==============================] - 0s 23ms/step - loss: 0.0329 - val_loss: 0.0299
Epoch 6/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0182 - val_loss: 0.0183
Epoch 7/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0089 - val_loss: 0.0125
Epoch 8/200
5/5 [==============================] - 0s 27ms/step - loss: 0.0057 - val_loss: 0.0100
Epoch 9/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0048 - val_loss: 0.0085
Epoch 10/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0040 - val_loss: 0.0071
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 18ms/step - loss: 3.5725e-05 - val_loss: 0.0015
Epoch 82/200
5/5 [==============================] - 0s 20ms/step - loss: 3.4993e-05 - val_loss: 0.0015
Epoch 83/200
5/5 [==============================] - 0s 20ms/step - loss: 3.3394e-05 - val_loss: 0.0015
Epoch 84/200
5/5 [==============================] - 0s 18ms/step - loss: 3.2647e-05 - val_loss: 0.0015
Epoch 85/200
5/5 [==============================] - 0s 20ms/step - loss: 3.0310e-05 - val_loss: 0.0015
Epoch 86/200
5/5 [==============================] - 0s 21ms/step - loss: 2.9756e-05 - val_loss: 0.0014
Epoch 87/200
5/5 [==============================] - 0s 18ms/step - loss: 2.8397e-05 - val_loss: 0.0014
Epoch 88/200
5/5 [==============================] - 0s 20ms/step - loss: 2.7786e-05 - val_loss: 0.0014
Epoch 89/200
5/5 [==============================] - 0s 20ms/step - loss: 2.7164e-05 - val_loss: 0.0014
Epoch 90/200
5/5 [==============================] - 0s 20ms/step - loss: 2.5543e-05 - 

5/5 [==============================] - 0s 19ms/step - loss: 5.0747e-06 - val_loss: 0.0012
Epoch 161/200
5/5 [==============================] - 0s 20ms/step - loss: 4.9222e-06 - val_loss: 0.0012
Epoch 162/200
5/5 [==============================] - 0s 22ms/step - loss: 5.0664e-06 - val_loss: 0.0012
Epoch 163/200
5/5 [==============================] - 0s 20ms/step - loss: 4.7147e-06 - val_loss: 0.0012
Epoch 164/200
5/5 [==============================] - 0s 20ms/step - loss: 4.8423e-06 - val_loss: 0.0012
Epoch 165/200
5/5 [==============================] - 0s 20ms/step - loss: 4.6718e-06 - val_loss: 0.0012
Epoch 166/200
5/5 [==============================] - 0s 20ms/step - loss: 4.4513e-06 - val_loss: 0.0012
Epoch 167/200
5/5 [==============================] - 0s 22ms/step - loss: 4.4036e-06 - val_loss: 0.0012
Epoch 168/200
5/5 [==============================] - 0s 20ms/step - loss: 4.6921e-06 - val_loss: 0.0012
Epoch 169/200
5/5 [==============================] - 0s 19ms/step - loss: 4.67

In [57]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 6.9703e-04 - 211ms/epoch - 211ms/step
tf.Tensor((0.999302975409275+0j), shape=(), dtype=complex128)


## Amplitude Damping Channel

In [58]:
#APPLY ERROR 
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = ampl_damp_error(1, density_matrix_noise_free[i], 0.5, 0.3)
    density_matrix_with_noise.append(single_data_with_noise)
#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

In [59]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

### MSE

In [60]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)), 
  tf.keras.layers.Dense(64, activation='relu'),  
  tf.keras.layers.Dense(64, activation='relu'),                                  
  tf.keras.layers.Dense(3)
  ])

In [61]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt,
              loss=loss_fn)

In [62]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val,y_val),batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 1s 71ms/step - loss: 0.3493 - val_loss: 0.3187
Epoch 2/200
5/5 [==============================] - 0s 20ms/step - loss: 0.2968 - val_loss: 0.2771
Epoch 3/200
5/5 [==============================] - 0s 23ms/step - loss: 0.2507 - val_loss: 0.2377
Epoch 4/200
5/5 [==============================] - 0s 26ms/step - loss: 0.2114 - val_loss: 0.2016
Epoch 5/200
5/5 [==============================] - 0s 22ms/step - loss: 0.1755 - val_loss: 0.1674
Epoch 6/200
5/5 [==============================] - 0s 23ms/step - loss: 0.1417 - val_loss: 0.1347
Epoch 7/200
5/5 [==============================] - 0s 22ms/step - loss: 0.1101 - val_loss: 0.1055
Epoch 8/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0822 - val_loss: 0.0787
Epoch 9/200
5/5 [==============================] - 0s 20ms/step - loss: 0.0579 - val_loss: 0.0558
Epoch 10/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0376 - val_loss: 0.0368
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 20ms/step - loss: 5.1336e-05 - val_loss: 2.9165e-04
Epoch 80/200
5/5 [==============================] - 0s 20ms/step - loss: 4.9946e-05 - val_loss: 2.9169e-04
Epoch 81/200
5/5 [==============================] - 0s 22ms/step - loss: 4.9049e-05 - val_loss: 2.8468e-04
Epoch 82/200
5/5 [==============================] - 0s 18ms/step - loss: 4.8213e-05 - val_loss: 2.7634e-04
Epoch 83/200
5/5 [==============================] - 0s 20ms/step - loss: 4.6480e-05 - val_loss: 2.7761e-04
Epoch 84/200
5/5 [==============================] - 0s 18ms/step - loss: 4.5884e-05 - val_loss: 2.7751e-04
Epoch 85/200
5/5 [==============================] - 0s 22ms/step - loss: 4.5283e-05 - val_loss: 2.6918e-04
Epoch 86/200
5/5 [==============================] - 0s 18ms/step - loss: 4.4221e-05 - val_loss: 2.7396e-04
Epoch 87/200
5/5 [==============================] - 0s 22ms/step - loss: 4.2847e-05 - val_loss: 2.7050e-04
Epoch 88/200
5/5 [==============================] 

5/5 [==============================] - 0s 20ms/step - loss: 1.1496e-05 - val_loss: 1.9202e-04
Epoch 156/200
5/5 [==============================] - 0s 26ms/step - loss: 1.1750e-05 - val_loss: 1.9320e-04
Epoch 157/200
5/5 [==============================] - 0s 20ms/step - loss: 1.1982e-05 - val_loss: 1.9312e-04
Epoch 158/200
5/5 [==============================] - 0s 22ms/step - loss: 1.1043e-05 - val_loss: 1.9113e-04
Epoch 159/200
5/5 [==============================] - 0s 24ms/step - loss: 1.1252e-05 - val_loss: 1.9303e-04
Epoch 160/200
5/5 [==============================] - 0s 22ms/step - loss: 1.0928e-05 - val_loss: 1.8852e-04
Epoch 161/200
5/5 [==============================] - 0s 21ms/step - loss: 1.0656e-05 - val_loss: 1.9042e-04
Epoch 162/200
5/5 [==============================] - 0s 18ms/step - loss: 1.0477e-05 - val_loss: 1.9006e-04
Epoch 163/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0622e-05 - val_loss: 1.8886e-04
Epoch 164/200
5/5 [=======================

In [63]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 1.1119e-04 - 193ms/epoch - 193ms/step
tf.Tensor((1.0007760737674012+0j), shape=(), dtype=complex128)


### INF

In [64]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
  ])

In [65]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 3s 507ms/step - loss: 0.4524 - val_loss: 0.3607
Epoch 2/200
5/5 [==============================] - 0s 28ms/step - loss: 0.2728 - val_loss: 0.2178
Epoch 3/200
5/5 [==============================] - 0s 24ms/step - loss: 0.1452 - val_loss: 0.0937
Epoch 4/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0483 - val_loss: 0.0411
Epoch 5/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0219 - val_loss: 0.0255
Epoch 6/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0132 - val_loss: 0.0145
Epoch 7/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0072 - val_loss: 0.0092
Epoch 8/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0041 - val_loss: 0.0077
Epoch 9/200
5/5 [==============================] - 0s 22ms/step - loss: 0.0034 - val_loss: 0.0070
Epoch 10/200
5/5 [==============================] - 0s 24ms/step - loss: 0.0029 - val_loss: 0.0061
Epoch 11/200
5/5 [

5/5 [==============================] - 0s 18ms/step - loss: 5.8907e-05 - val_loss: 9.6912e-04
Epoch 81/200
5/5 [==============================] - 0s 18ms/step - loss: 5.7330e-05 - val_loss: 9.6952e-04
Epoch 82/200
5/5 [==============================] - 0s 20ms/step - loss: 5.7207e-05 - val_loss: 9.7364e-04
Epoch 83/200
5/5 [==============================] - 0s 18ms/step - loss: 5.4011e-05 - val_loss: 9.7062e-04
Epoch 84/200
5/5 [==============================] - 0s 18ms/step - loss: 5.4836e-05 - val_loss: 9.6599e-04
Epoch 85/200
5/5 [==============================] - 0s 21ms/step - loss: 5.1913e-05 - val_loss: 9.6678e-04
Epoch 86/200
5/5 [==============================] - 0s 20ms/step - loss: 5.2291e-05 - val_loss: 9.6951e-04
Epoch 87/200
5/5 [==============================] - 0s 18ms/step - loss: 4.8949e-05 - val_loss: 9.6598e-04
Epoch 88/200
5/5 [==============================] - 0s 22ms/step - loss: 4.9653e-05 - val_loss: 9.6772e-04
Epoch 89/200
5/5 [==============================] 

5/5 [==============================] - 0s 25ms/step - loss: 1.1979e-05 - val_loss: 9.3094e-04
Epoch 157/200
5/5 [==============================] - 0s 22ms/step - loss: 1.1077e-05 - val_loss: 9.3179e-04
Epoch 158/200
5/5 [==============================] - 0s 20ms/step - loss: 1.1387e-05 - val_loss: 9.3566e-04
Epoch 159/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0880e-05 - val_loss: 9.3050e-04
Epoch 160/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0738e-05 - val_loss: 9.2606e-04
Epoch 161/200
5/5 [==============================] - 0s 26ms/step - loss: 1.0124e-05 - val_loss: 9.3340e-04
Epoch 162/200
5/5 [==============================] - 0s 20ms/step - loss: 1.0490e-05 - val_loss: 9.3353e-04
Epoch 163/200
5/5 [==============================] - 0s 23ms/step - loss: 1.0055e-05 - val_loss: 9.3516e-04
Epoch 164/200
5/5 [==============================] - 0s 24ms/step - loss: 1.0580e-05 - val_loss: 9.3542e-04
Epoch 165/200
5/5 [=======================

In [66]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 3.9188e-04 - 216ms/epoch - 216ms/step
tf.Tensor((0.9996081326010282+0j), shape=(), dtype=complex128)


## Depolarizing Channel

In [67]:
#APPLY ERROR 
density_matrix_with_noise = []
for i in range(len(data)):
    single_data_with_noise = depolar_error(1, density_matrix_noise_free[i], 0.3)
    density_matrix_with_noise.append(single_data_with_noise)
#COMPUTE_BLOCH VECTOR WITH NOISE
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

In [68]:
#Building the training, validation and test set 

x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:50], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:50], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]

#Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

### MSE

In [69]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)), 
  tf.keras.layers.Dense(64, activation='relu'),  
  tf.keras.layers.Dense(64, activation='relu'),                                  
  tf.keras.layers.Dense(3)
  ])

In [70]:
# Define Loss
loss_fn = tf.keras.losses.mse
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt,
              loss=loss_fn)

In [71]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val,y_val),batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 94ms/step - loss: 0.3360 - val_loss: 0.3226
Epoch 2/200
5/5 [==============================] - 0s 22ms/step - loss: 0.2966 - val_loss: 0.2889
Epoch 3/200
5/5 [==============================] - 0s 22ms/step - loss: 0.2611 - val_loss: 0.2561
Epoch 4/200
5/5 [==============================] - 0s 22ms/step - loss: 0.2275 - val_loss: 0.2259
Epoch 5/200
5/5 [==============================] - 0s 24ms/step - loss: 0.1958 - val_loss: 0.1968
Epoch 6/200
5/5 [==============================] - 0s 24ms/step - loss: 0.1650 - val_loss: 0.1680
Epoch 7/200
5/5 [==============================] - 0s 28ms/step - loss: 0.1361 - val_loss: 0.1404
Epoch 8/200
5/5 [==============================] - 0s 29ms/step - loss: 0.1085 - val_loss: 0.1139
Epoch 9/200
5/5 [==============================] - 0s 28ms/step - loss: 0.0838 - val_loss: 0.0897
Epoch 10/200
5/5 [==============================] - 0s 26ms/step - loss: 0.0629 - val_loss: 0.0680
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 20ms/step - loss: 7.9886e-05 - val_loss: 3.9657e-04
Epoch 80/200
5/5 [==============================] - 0s 20ms/step - loss: 7.5292e-05 - val_loss: 3.9726e-04
Epoch 81/200
5/5 [==============================] - 0s 19ms/step - loss: 7.3439e-05 - val_loss: 3.8682e-04
Epoch 82/200
5/5 [==============================] - 0s 22ms/step - loss: 7.8119e-05 - val_loss: 3.8630e-04
Epoch 83/200
5/5 [==============================] - 0s 21ms/step - loss: 7.0523e-05 - val_loss: 3.7909e-04
Epoch 84/200
5/5 [==============================] - 0s 24ms/step - loss: 7.1819e-05 - val_loss: 3.7230e-04
Epoch 85/200
5/5 [==============================] - 0s 20ms/step - loss: 6.6960e-05 - val_loss: 3.8375e-04
Epoch 86/200
5/5 [==============================] - 0s 20ms/step - loss: 6.8418e-05 - val_loss: 3.8432e-04
Epoch 87/200
5/5 [==============================] - 0s 23ms/step - loss: 6.4600e-05 - val_loss: 3.7259e-04
Epoch 88/200
5/5 [==============================] 

5/5 [==============================] - 0s 25ms/step - loss: 1.7648e-05 - val_loss: 2.9212e-04
Epoch 156/200
5/5 [==============================] - 0s 20ms/step - loss: 1.6622e-05 - val_loss: 2.8925e-04
Epoch 157/200
5/5 [==============================] - 0s 26ms/step - loss: 1.6592e-05 - val_loss: 2.8710e-04
Epoch 158/200
5/5 [==============================] - 0s 20ms/step - loss: 1.5987e-05 - val_loss: 2.8645e-04
Epoch 159/200
5/5 [==============================] - 0s 18ms/step - loss: 1.6566e-05 - val_loss: 2.8221e-04
Epoch 160/200
5/5 [==============================] - 0s 20ms/step - loss: 1.5919e-05 - val_loss: 2.8709e-04
Epoch 161/200
5/5 [==============================] - 0s 20ms/step - loss: 1.5465e-05 - val_loss: 2.8691e-04
Epoch 162/200
5/5 [==============================] - 0s 18ms/step - loss: 1.4815e-05 - val_loss: 2.8204e-04
Epoch 163/200
5/5 [==============================] - 0s 21ms/step - loss: 1.5202e-05 - val_loss: 2.7938e-04
Epoch 164/200
5/5 [=======================

In [72]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 1.5879e-04 - 242ms/epoch - 242ms/step
tf.Tensor((0.9984214997982033+0j), shape=(), dtype=complex128)


### INF

In [73]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3),
  tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) 
  ])

In [74]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(optimizer=adam_opt, 
              loss=infidelity)

# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 141ms/step - loss: 0.5151 - val_loss: 0.3175
Epoch 2/200
5/5 [==============================] - 0s 54ms/step - loss: 0.3054 - val_loss: 0.1790
Epoch 3/200
5/5 [==============================] - 0s 70ms/step - loss: 0.1316 - val_loss: 0.0697
Epoch 4/200
5/5 [==============================] - 0s 36ms/step - loss: 0.0409 - val_loss: 0.0326
Epoch 5/200
5/5 [==============================] - 0s 60ms/step - loss: 0.0276 - val_loss: 0.0268
Epoch 6/200
5/5 [==============================] - 0s 56ms/step - loss: 0.0199 - val_loss: 0.0190
Epoch 7/200
5/5 [==============================] - 0s 36ms/step - loss: 0.0117 - val_loss: 0.0122
Epoch 8/200
5/5 [==============================] - 0s 38ms/step - loss: 0.0061 - val_loss: 0.0086
Epoch 9/200
5/5 [==============================] - 0s 34ms/step - loss: 0.0041 - val_loss: 0.0068
Epoch 10/200
5/5 [==============================] - 0s 39ms/step - loss: 0.0032 - val_loss: 0.0059
Epoch 11/200
5/5 [

5/5 [==============================] - 0s 24ms/step - loss: 4.2164e-05 - val_loss: 0.0011
Epoch 82/200
5/5 [==============================] - 0s 42ms/step - loss: 3.9818e-05 - val_loss: 0.0011
Epoch 83/200
5/5 [==============================] - 0s 46ms/step - loss: 3.8642e-05 - val_loss: 0.0011
Epoch 84/200
5/5 [==============================] - 0s 36ms/step - loss: 3.9561e-05 - val_loss: 0.0011
Epoch 85/200
5/5 [==============================] - 0s 34ms/step - loss: 3.7347e-05 - val_loss: 0.0011
Epoch 86/200
5/5 [==============================] - 0s 34ms/step - loss: 3.6267e-05 - val_loss: 0.0011
Epoch 87/200
5/5 [==============================] - 0s 30ms/step - loss: 3.4982e-05 - val_loss: 0.0011
Epoch 88/200
5/5 [==============================] - 0s 28ms/step - loss: 3.5005e-05 - val_loss: 0.0011
Epoch 89/200
5/5 [==============================] - 0s 56ms/step - loss: 3.3480e-05 - val_loss: 0.0011
Epoch 90/200
5/5 [==============================] - 0s 38ms/step - loss: 3.2406e-05 - 

Epoch 160/200
5/5 [==============================] - 0s 28ms/step - loss: 5.7113e-06 - val_loss: 0.0010
Epoch 161/200
5/5 [==============================] - 0s 21ms/step - loss: 5.7292e-06 - val_loss: 9.9865e-04
Epoch 162/200
5/5 [==============================] - 0s 22ms/step - loss: 5.6696e-06 - val_loss: 9.9899e-04
Epoch 163/200
5/5 [==============================] - 0s 20ms/step - loss: 5.5742e-06 - val_loss: 9.9546e-04
Epoch 164/200
5/5 [==============================] - 0s 21ms/step - loss: 5.5325e-06 - val_loss: 9.9333e-04
Epoch 165/200
5/5 [==============================] - 0s 22ms/step - loss: 5.3728e-06 - val_loss: 9.9920e-04
Epoch 166/200
5/5 [==============================] - 0s 21ms/step - loss: 5.3978e-06 - val_loss: 9.9756e-04
Epoch 167/200
5/5 [==============================] - 0s 20ms/step - loss: 5.1129e-06 - val_loss: 9.9641e-04
Epoch 168/200
5/5 [==============================] - 0s 21ms/step - loss: 5.2154e-06 - val_loss: 9.9447e-04
Epoch 169/200
5/5 [=============

In [75]:
score = model.evaluate(x_test,  y_test, verbose=2)

#save the model predictions in a tensor
y_prediction = model(x_test)

#CHANGE TYPE OF PREDICTION IN COMPLEX ONE
y_prediction = tf.cast(y_prediction, tf.complex128)

#CHANGE TYPE OF IDEAL CASE IN COMPLEX ONE
y_test = tf.cast(y_test, tf.complex128)

fidelities = []

for i in range(len(y_prediction)):
  den_mat = density_matrix_from_bloch_vector(y_prediction[i])
  den_mat = np.asarray(den_mat)


  den_mat_id = density_matrix_from_bloch_vector(y_test[i])
  den_mat_id = np.asarray(den_mat_id)


  fidelity = fidelity_function(den_mat_id, den_mat)
  fidelities.append(fidelity)

print(tf.math.reduce_mean(fidelities))

1/1 - 0s - loss: 2.2932e-04 - 273ms/epoch - 273ms/step
tf.Tensor((0.9997706817103381+0j), shape=(), dtype=complex128)
